In [ ]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from datetime import date


"""data has been saved as an xml file"""
import time
timestr = time.strftime("%Y%m%d-%H%M%S")


#import file name and string with date and time and save as variable
qld_fire_data = 'data/qld_data/qld_fire_data_'+timestr+'.xml'

#add the data from the website 
URL = "https://www.qfes.qld.gov.au/data/alerts/bushfireAlert.xml"

#The binary mode (wb) is required to write the actual content of the XML page to your external file in the original format.
#will get the current up to date details from website
response = requests.get(URL)
with open(qld_fire_data, 'wb') as file:
    file.write(response.content)

#parse through the data    
mytree = ET.parse(qld_fire_data) 
myroot = mytree.getroot()

geo_list=[]
for geo in myroot.findall('.//{http://www.georss.org/georss}point'):
    geo_list.append(geo.text)

co_ordinates_split =[]
for value in geo_list:
    co_ordinates_split.append(value.split())

qld_latitude_list = []
qld_longitude_list = []
for co_ordinates in co_ordinates_split:
    qld_latitude_list.append(co_ordinates[0])
    qld_longitude_list.append(co_ordinates[1])


today = date.today()
qld_date = today.strftime("%d/%m/%Y")

qld_df = pd.DataFrame({
    'Latitude': qld_latitude_list,
    'Longitude': qld_longitude_list,
    'State': 'QLD',
    'Report Date': qld_date
})

qld_df.to_csv("data/fire_data.csv", mode = 'a', index = False , header = False)

#remove duplicate values
fire_data_df = pd.read_csv('data/fire_data.csv')

#drop the duplicates out of the data frame
fire_data_df_no_duplicates = fire_data_df.drop_duplicates()

#create a new csv file with the dropped duplicate values
fire_data_df_no_duplicates.to_csv('data/fire_data_df_no_duplicates.csv',index = False)
display(fire_data_df_no_duplicates)